In [1]:
from langchain_community.chat_models import ChatZhipuAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

In [2]:
import os
os.environ["ZHIPUAI_API_KEY"] = "6771943d3790217c139b3809aec6858f.np9HoDC4J6JIyyQh"

In [3]:
chat = ChatZhipuAI(
    model="glm-4-air",
    temperature=0.2,
)

In [4]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

# 定义关于最小访问次数的响应方案
min_visits_schema = ResponseSchema(name="N_c_min",
                                   type="list[integer]",
                                   description="类别的最小访问次数，按照景点、餐饮、酒店的顺序以list的形式给出。默认为[0, 2, 2]。例如餐饮最小数量设置为3代表需要早午晚餐，美食之旅则改为4等。")

# 定义关于最大访问次数的响应方案
max_visits_schema = ResponseSchema(name="N_c_max",
                                   type="list[integer]",
                                   description="类别的最大访问次数，按照景点、餐饮、酒店的顺序以list的形式给出。默认为[10, 2, 2]。例如美食相关探店需要大于等于5的设置。")

# 定义关于餐饮最小时间间隔的响应方案
min_restaurant_gap_schema = ResponseSchema(name="use_min_restaurant_gap",
                                           type="float",
                                           description="餐饮类别的最小时间间隔约束，范围在[1, 3]之间，默认是3。更小的数值适用于需要频繁探店或体验美食之旅的情况。")

# 定义必须访问的POI列表的响应方案
mandatory_pois_schema = ResponseSchema(name="poi_id_list",
                                       type="list[string]",
                                       description="用户必须访问的POI列表，按照用户偏好在你的知识库中进行筛选，数量上达到平均每天选择1-2个景点。")

# 定义路线数的响应方案
route_num_schema = ResponseSchema(name="route_num",
                                  type="integer",
                                  description="路线数，即旅行天数，默认为3天。")

# 定义起始点的响应方案
start_end_poi_schema = ResponseSchema(name="start_end_poi",
                                      type="string",
                                      description="单路线起始点（酒店），默认为null，用户可指定修改。")

# 定义每天开始时间的响应方案
start_day_time_schema = ResponseSchema(name="start_day_time",
                                       type="string",
                                       description="每天的开始时间，默认为09:00:00。根据需要设定更早或更晚的时间，以适应休闲或紧凑的旅游风格。")

# 定义单条路线最大时间的响应方案
max_time_schema = ResponseSchema(name="plan_max_time",
                                 type="integer",
                                 description="单条路线最大时间，默认为12小时。根据需要调整时间长度，适应更休闲或更紧凑的旅行计划。")

# 定义紧凑度权重的响应方案
tightness_weight_schema = ResponseSchema(name="tightness_w",
                                         type="float",
                                         description="紧凑度权重，设置在0.5至1.5之间。根据用户需求调整，更高值适合休闲体验，较低值适合紧凑旅行。")

# 定义用户预算的响应方案
user_budget_schema = ResponseSchema(name="user_budget",
                                    type="float",
                                    description="用户预算，以1e9为默认值，表示没有预算限制。")

# 定义用户不考虑的POI列表的响应方案
not_poi_list_schema = ResponseSchema(name="not_poi_list",
                                     type="list[string]",
                                     description="用户不考虑访问的POI列表，根据个人偏好排除某些POI，默认为空列表。")

# 将所有方案添加到列表中
response_schemas = [
    min_visits_schema, max_visits_schema, min_restaurant_gap_schema,
    mandatory_pois_schema, route_num_schema, start_end_poi_schema,
    start_day_time_schema, max_time_schema, tightness_weight_schema,
    user_budget_schema, not_poi_list_schema
]

# 创建一个结构化输出解析器
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# 获取并打印格式化指令
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"N_c_min": list[integer]  // 类别的最小访问次数，按照景点、餐饮、酒店的顺序以list的形式给出。默认为[0, 2, 2]。例如餐饮最小数量设置为3代表需要早午晚餐，美食之旅则改为4等。
	"N_c_max": list[integer]  // 类别的最大访问次数，按照景点、餐饮、酒店的顺序以list的形式给出。默认为[10, 2, 2]。例如美食相关探店需要大于等于5的设置。
	"use_min_restaurant_gap": float  // 餐饮类别的最小时间间隔约束，范围在[1, 3]之间，默认是3。更小的数值适用于需要频繁探店或体验美食之旅的情况。
	"poi_id_list": list[string]  // 用户必须访问的POI列表，按照用户偏好在你的知识库中进行筛选，数量上达到平均每天选择1-2个景点。
	"route_num": integer  // 路线数，即旅行天数，默认为3天。
	"start_end_poi": string  // 单路线起始点（酒店），默认为null，用户可指定修改。
	"start_day_time": string  // 每天的开始时间，默认为09:00:00。根据需要设定更早或更晚的时间，以适应休闲或紧凑的旅游风格。
	"plan_max_time": integer  // 单条路线最大时间，默认为12小时。根据需要调整时间长度，适应更休闲或更紧凑的旅行计划。
	"tightness_w": float  // 紧凑度权重，设置在0.5至1.5之间。根据用户需求调整，更高值适合休闲体验，较低值适合紧凑旅行。
	"user_budget": float  // 用户预算，以1e9为默认值，表示没有预算限制。
	"not_poi_list": list[string]  // 用户不考虑访问的POI列表，根据个人偏好排除某些POI，默认为空列表。
}
```


In [5]:
messages = [
    AIMessage(content="你好"),
    SystemMessage(content=f"你是一个人类语言与行程规划算法之间的翻译者，能够专业理解提取用户的旅行偏好并对应到算法的输入上，你的工作是根据你的知识一次理解用户的需求（不可询问额外信息），根据单轮用户的需求给出合理的函数输入参数，所有POI均需要给出完整准确的信息，不能缩写，使用中文，输出不能包含任何注释。\n {format_instructions}"),
    HumanMessage(content="我的两天春季计划包括游览五大道，体验海河的风光和春天的氛围。"),
]

In [6]:
response = chat.invoke(messages)
print(response)
print(response.content)

content='```json\n{\n\t"N_c_min": [1, 2, 1],\n\t"N_c_max": [5, 3, 2],\n\t"use_min_restaurant_gap": 2.5,\n\t"poi_id_list": ["五大道", "海河"],\n\t"route_num": 2,\n\t"start_end_poi": null,\n\t"start_day_time": "09:00:00",\n\t"plan_max_time": 10,\n\t"tightness_w": 1.0,\n\t"user_budget": 1e9,\n\t"not_poi_list": []\n}\n```' response_metadata={'token_usage': {'completion_tokens': 129, 'prompt_tokens': 565, 'total_tokens': 694}, 'model_name': 'glm-4-air', 'finish_reason': 'stop'} id='run-ba571ab6-cdd0-4d3e-84a0-88ed80cef543-0'
```json
{
	"N_c_min": [1, 2, 1],
	"N_c_max": [5, 3, 2],
	"use_min_restaurant_gap": 2.5,
	"poi_id_list": ["五大道", "海河"],
	"route_num": 2,
	"start_end_poi": null,
	"start_day_time": "09:00:00",
	"plan_max_time": 10,
	"tightness_w": 1.0,
	"user_budget": 1e9,
	"not_poi_list": []
}
```


In [7]:
output_dict = output_parser.parse(response.content)
output_dict

{'N_c_min': [1, 2, 1],
 'N_c_max': [5, 3, 2],
 'use_min_restaurant_gap': 2.5,
 'poi_id_list': ['五大道', '海河'],
 'route_num': 2,
 'start_end_poi': None,
 'start_day_time': '09:00:00',
 'plan_max_time': 10,
 'tightness_w': 1.0,
 'user_budget': 1000000000.0,
 'not_poi_list': []}

In [12]:
import dataset_config as dataset_config

dataConfig = dataset_config.DatasetConfig()

database = dataConfig.database
poi_dict = dataConfig.poi_dict
database_function = dataConfig.database_func
poi_cate_dict = dataConfig.poi_cate_dict

def getIdsbyNames(poi_name_list:list[str], poi_cate:str):
    database.connect()
    poi_name_list = ','.join(poi_name_list)
    poi_list = database_function['getIdbyName'](poi_name_list, poi_cate_dict[poi_cate])
    poi_id_list = [i[0] for i in poi_list]
    poi_id_list = list(set(poi_id_list))
    database.close()
    return poi_id_list

In [13]:
from GRASP import GRASP
plan_entity = GRASP(
    N_c_min=[0, output_dict.get('N_c_min')[1], 2],
    N_c_max=[10, output_dict.get('N_c_max')[1], 2],
    maxIterations=1,
    poi_id_list=getIdsbyNames(output_dict.get("poi_id_list"), 'attraction'),
    route_num=output_dict.get("route_num"),
    not_poi_list=getIdsbyNames(output_dict.get("not_poi_list"), 'attraction'),
    use_min_restaurant_gap=output_dict.get("use_min_restaurant_gap"),
    start_day_time=output_dict.get("start_day_time"),
    plan_max_time=output_dict.get("plan_max_time"),
    tightness_w=output_dict.get("tightness_w"),
    user_budget=output_dict.get("user_budget"),
    start_end_poi=getIdsbyNames(output_dict.get("start_end_poi"), 'hotel')[0] if output_dict.get("start_end_poi") is not None else None
)

results, st, wt, ts, tt, ds = plan_entity.GRASP()
print(results)

[[[1035, 2, '如家商旅酒店(天津五大道游客中心外国语大学店)', '天津', 4.8, '￥472', 39.115555, 117.21031, 0, 29, '位置交通 位置很好,房间床品也很干净,五大道里面', '{"名称":"如家商旅酒店(天津五大道游客中心外国语大学店)","分数":4.8,"人气排名":6,"图片链接":"https:\\/\\/userimg.qunarzz.com\\/imgs\\/202404\\/27\\/C.UecTTSvVlVYWeQZgs720.jpg","简介":"位置交通 位置很好,房间床品也很干净,五大道里面","地址":"舒适型|天津和平区睦南道72号","经度":117.210309,"纬度":39.115554,"价格":null,"点评数量":29}', '0101000020E61000007C43E1B3754D5D4004763579CA8E4340', '舒适型|天津和平区睦南道72号', [0, None], [86340, None], 0, [0, None], [86340, None]], [8, 0, '五大道', '天津', 4.6, '五大道位于天津和平区，以区域内五个主要的道路命名，实际有二十多条街道。清朝遗老、北洋政府里很多官员名流都曾在这里寓居，一栋栋优美典雅的洋房仿佛在诉说着津门曾经的风云故事。\n\n五大道总述目前来到五大道游览，主要就是观看街道上2000多所花园式的欧式建筑，其中名人故居就有三百多处，张学良、顾维钧、爱新觉罗载振等众多清末民初的名流都在这里留下了名字。这里的建筑整体是欧式，但形式多样各有特色。街区优雅浪漫，不少影视剧组来此取景，拍婚纱照的也爱把五大道的小洋房当背景。\n游玩方式初到五大道，如果摸不清东南西北，可以先到长沙路101号的民园广场，看看曾经的欧式体育馆，这里也相当于是五大道的集散中心。广场附近的游客中心里，可以拿一份地图（也有付费购买的更清晰版），上面标注了五大道很多故居、建筑的所在地，根据地图参观游玩即可。\n参观重点五大道内，很多的景点值得参观。疙瘩楼在河北路，墙上一粒一粒烧过火的砖形成的疙瘩已经十分特别，而且还是一座瓷片贴满的洋房；马场道117号的天津外国语学院是电视剧《金粉世家》的取景地；清朝庆亲王载振的公馆建筑中西合璧，在重庆道

In [15]:
[result[poi_dict['poi_name']] for result_day in results for result in result_day]

['如家商旅酒店(天津五大道游客中心外国语大学店)',
 '五大道',
 '五大道小洋楼',
 '大鹅经典俄式西餐',
 '海河文化广场',
 '解放桥',
 'La Fleur用心(和平店)',
 '华竹食府(南海路店)',
 '如家商旅酒店(天津五大道游客中心外国语大学店)',
 '如家商旅酒店(天津五大道游客中心外国语大学店)',
 '天津科学技术馆',
 '津菜典藏(人民公园店)',
 '人民公园',
 '海河',
 '昱德来菜馆(河西店)',
 '粤之园(澳门路店)',
 '如家商旅酒店(天津五大道游客中心外国语大学店)']

In [1]:
import pandas as pd

day_pois = pd.read_csv('../data/Beijing_case_dataset - Day&POIs.csv')
day_pois.head()

,day,poi,特色,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,6,天安门广场 > 故宫博物院 > 什刹海 > 南锣鼓巷\n天坛 > 前门大街\n八达岭长城 >...,经典、全面,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,南锣鼓巷 > 五道营胡同 > 国子监\n八大胡同 > 大栅栏\n798艺术区 > 奥林匹克公...,土著、原汁原味,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,天安门广场 > 故宫博物院 > 南锣鼓巷 > 后海公园\n颐和园 > 清华大学 > 北京大学...,穷游、学生、地标,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,天安门广场 > 故宫博物院 > 什刹海 > 南锣鼓巷\n天坛 > 前门大街\n八达岭长城\n...,古迹、繁华,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,南锣鼓巷 > 恭王府 > 什刹海\n国子监 > 五道营胡同\n八达岭长城\n富国海底世界,深度、小长假,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
# drop all Nan column
day_pois = day_pois[['day', 'poi', '特色']]
day_pois.head()

,day,poi,特色
0,6,天安门广场 > 故宫博物院 > 什刹海 > 南锣鼓巷\n天坛 > 前门大街\n八达岭长城 >...,经典、全面
1,3,南锣鼓巷 > 五道营胡同 > 国子监\n八大胡同 > 大栅栏\n798艺术区 > 奥林匹克公...,土著、原汁原味
2,4,天安门广场 > 故宫博物院 > 南锣鼓巷 > 后海公园\n颐和园 > 清华大学 > 北京大学...,穷游、学生、地标
3,5,天安门广场 > 故宫博物院 > 什刹海 > 南锣鼓巷\n天坛 > 前门大街\n八达岭长城\n...,古迹、繁华
4,4,南锣鼓巷 > 恭王府 > 什刹海\n国子监 > 五道营胡同\n八达岭长城\n富国海底世界,深度、小长假


In [ ]:
# poi 列按照\n分割为两列

In [3]:
import json
import random
import time
random.seed(42)
current_time = time.strftime("%Y%m%d%H%M%S")
catering_requirements = [
    {
        "feature": "常规",
        "N_c_min": [1, 2, 2],
        "N_c_max": [10, 3, 2],
        "use_min_restaurant_gap": 10800
    },
    {
        "feature": "美食",
        "N_c_min": [1, 3, 2],
        "N_c_max": [10, 6, 2],
        "use_min_restaurant_gap": 3600
    }
]
time_requirements = [
    {
        "feature": "慢节奏",
        "start_day_time": "09:00:00",
        "plan_max_time": 12,
        "tightness_w": 1.5
    },
    {
        "feature": "正常",
        "start_day_time": "09:00:00",
        "plan_max_time": 12,
        "tightness_w": 1
    },
    {
        "feature": "快节奏",
        "start_day_time": "09:00:00",
        "plan_max_time": 12,
        "tightness_w": 0.5
    },
    {
        "feature": "时间休闲",
        "start_day_time": "10:30:00",
        "plan_max_time": 10,
        "tightness_w": 1
    },
    {
        "feature": "时间高强度",
        "start_day_time": "08:30:00",
        "plan_max_time": 14,
        "tightness_w": 1
    },
    {
        "feature": "特种兵式",
        "start_day_time": "08:00:00",
        "plan_max_time": 14,
        "tightness_w": 0.5
    }
]

day_poi_case = []
for index, row in day_pois.iterrows():
    every_day = row["poi"].split('\n')
    route_num = row["day"]
    for i in range(20):
        poi_name_list = []
        for every_day_pois in every_day:
            every_day_pois_list = every_day_pois.split(' > ')
            #random choice from every_day_pois_list
            random_k = random.randint(0, len(every_day_pois_list) - 1)
            poi_name_list.extend(random.sample(every_day_pois_list, k=random_k))
        for catering_requirement in catering_requirements:
            for time_requirement in time_requirements:    
                day_poi_case.append({
                    "poi_feature": row['特色'],
                    "route_num": route_num,
                    "poi_name_list": poi_name_list,
                    "catering_feature": catering_requirement['feature'],
                    "N_c_min": catering_requirement['N_c_min'],
                    "N_c_max": catering_requirement['N_c_max'],
                    "use_min_restaurant_gap": catering_requirement['use_min_restaurant_gap'],
                    "time_feature": time_requirement['feature'],
                    "start_day_time": time_requirement['start_day_time'],
                    "plan_max_time": time_requirement['plan_max_time'],
                    "tightness_w": time_requirement['tightness_w']
                })
        
#day_poi_case save to json file
with open(f'../data/day_poi_case_{current_time}.json', 'w', encoding='utf-8') as f:
    json.dump(day_poi_case, f, ensure_ascii=False, indent=4)